In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,ReLU,GlobalAveragePooling1D
from tensorflow.keras.layers import BatchNormalization,Dropout,Embedding,Activation,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import keras
 
from tensorflow.keras.layers import BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

METRICS=[
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
!pip install split-folders

In [ ]:
import splitfolders
import os
#path = "/kaggle/input/kvasir-final-processing/kvasir_bilateral_filtered"
path="/kaggle/input/the-kvasir-dataset/kvasir-dataset-v2"
splitfolders.ratio(path,seed=1337, output="Train_test_splited", ratio=(0.7, 0.15, 0.15))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_gen = ImageDataGenerator(rescale= 1/255.0, 
                                rotation_range=10,  
                                zoom_range = 0.1, 
                                horizontal_flip=True,
                                vertical_flip=True,
                                 fill_mode='nearest' 
                                )
test_gen=ImageDataGenerator(rescale= 1/255.0)
train_path="/kaggle/working/Train_test_splited/train"
val_path="/kaggle/working/Train_test_splited/val"
test_path="/kaggle/working/Train_test_splited/test"
train = train_image_gen.flow_from_directory(train_path,
                                      target_size=(256,256),
                                      color_mode='rgb',
                                      class_mode="categorical", # used for Sequential Model
                                      batch_size=32,
                                      shuffle=True            #do not shuffle data
                                     )

val = train_image_gen.flow_from_directory(val_path,
                                    target_size=(256,256),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=True
                                   )
test = test_gen.flow_from_directory(test_path,
                                     target_size=(256,256),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=32,
                                     shuffle= False
                                    )

In [ ]:
def plot_images(generator, num_images=9):
    images, labels = next(generator)  # take one batch
    class_names = list(generator.class_indices.keys())

    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        ax = plt.subplot(3, 3, i + 1)
        
        
        plt.imshow(images[i],cmap='gray')
        label_index = np.argmax(labels[i])
        plt.title(class_names[label_index])
        plt.axis("off")
    plt.show()
plot_images(train)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, ops
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense,
    BatchNormalization, ReLU, Add, Multiply, GlobalAveragePooling2D
)


# Hyperparameters
IMG_SIZE = 256
NUM_CLASSES = 8
PATCH_SIZE = 4
NUM_FILTERS = 256
DEPTH = 10
KERNEL_SIZE = 4
DROPOUT_RATE = 0.2
HEADS = 4
MLP_DIM = 256
class GELU(layers.Layer):
    def call(self, inputs):
        return tf.nn.gelu(inputs)

# ConvMixer Block
  # Depth of the Co

def convmixer_block(inputs, num_filters, kernel_size):
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(inputs)
    x = GELU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, inputs])  # Residual connection
    
    x = layers.Conv2D(num_filters, kernel_size=1, padding="same")(x)
    x = GELU()(x)
    x = layers.BatchNormalization()(x)
    return x
def convmixer_whole_block(inputs, num_filters, patch_size):
    x = layers.Conv2D(num_filters, kernel_size=patch_size, strides=patch_size, padding="same")(inputs)
    x = GELU()(x)
    x = layers.BatchNormalization()(x)
    for _ in range(8):
        x = convmixer_block(x, num_filters, kernel_size=5)
        
    return x
        

# External Attention Block
def external_attention(x, dim, num_heads, dim_coefficient=4, attention_dropout=0, projection_dropout=0):
    _, num_patch, channel = x.shape[-3:]
    assert dim % num_heads == 0
    num_heads = num_heads * dim_coefficient

    x = layers.Dense(dim * dim_coefficient)(x)
    x = ops.reshape(x, (-1, num_patch, num_heads, dim * dim_coefficient // num_heads))
    x = ops.transpose(x, axes=[0, 2, 1, 3])
    
    attn = layers.Dense(dim // dim_coefficient)(x)
    attn = layers.Softmax(axis=2)(attn)
    attn = layers.Lambda(
        lambda attn: ops.divide(
            attn,
            ops.convert_to_tensor(1e-9) + ops.sum(attn, axis=-1, keepdims=True),
        )
    )(attn)
    attn = layers.Dropout(attention_dropout)(attn)

    x = layers.Dense(dim * dim_coefficient // num_heads)(attn)
    x = ops.transpose(x, axes=[0, 2, 1, 3])
    x = ops.reshape(x, [-1, num_patch, dim * dim_coefficient])

    x = layers.Dense(dim)(x)
    x = layers.Dropout(projection_dropout)(x)
    return x

# Build the model
def build_model(img_size, num_classes, patch_size, num_filters, depth, kernel_size, heads, mlp_dim, dropout_rate):
    inputs = tf.keras.Input((img_size, img_size, 3))
    
    
    
    cm1 = convmixer_whole_block(inputs, num_filters=256, patch_size=4)
    
    cm2 = convmixer_whole_block(inputs, num_filters=256, patch_size=8)
    #cm3 = convmixer_whole_block(inputs, num_filters=256, patch_size=8)
    
    up3 = layers.UpSampling2D(size=(2, 2))(cm2)
    #up3 = layers.UpSampling2D(size=(4, 4))(cm3)
    #print(cm1.shape, up2.shape,up3.shape)
    
    # Element-wise aggregation
    x = layers.Add()([cm1, up3])
    x=layers.MaxPooling2D(pool_size=2, strides=2, padding="same")(x)
    
    
    # Flatten and Project patches for the attention
    B, H, W, C = x.shape
    x = layers.Reshape((H*W, C))(x)
    
    #for _ in range(6):
     #  x1 = transformer_encoder(x1, num_heads=4, mlp_dim=256, dropout_rate=0.1)
    # External Attention
    x = external_attention(x, num_filters, num_heads=heads, attention_dropout=dropout_rate, projection_dropout=dropout_rate)
    #x=layers.Add()([x1,x2])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # Classifier
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs, outputs, name="ConvMixer_EANet")
    return model

# Compile the model
model = build_model(
    img_size=IMG_SIZE,
    num_classes=NUM_CLASSES,
    patch_size=PATCH_SIZE,
    num_filters=NUM_FILTERS,
    depth=DEPTH,
    kernel_size=KERNEL_SIZE,
    heads=HEADS,
    mlp_dim=MLP_DIM,
    dropout_rate=DROPOUT_RATE
)
#optimizer = tf.keras.optimizers.AdamW(
#        learning_rate=0.0001, weight_decay=0.0001)
#model.compile(
#    optimizer=optimizer,
#    loss='categorical_crossentropy',
#    metrics=METRICS
#)

# Model summary
model.summary()

In [ ]:
def sample_beta_distribution(size, concentration=0.2):
    gamma_1 = tf.random.gamma(shape=[size], alpha=concentration)
    gamma_2 = tf.random.gamma(shape=[size], alpha=concentration)
    return gamma_1 / (gamma_1 + gamma_2)

def mixup(batch_x, batch_y, alpha=0.2):
    lam = sample_beta_distribution(batch_x.shape[0], alpha)
    lam_x = lam[:, tf.newaxis, tf.newaxis, tf.newaxis]
    lam_y = lam[:, tf.newaxis]
    index = tf.random.shuffle(tf.range(batch_x.shape[0]))
    mixed_x = batch_x * lam_x + tf.gather(batch_x, index) * (1 - lam_x)
    mixed_y = batch_y * lam_y + tf.gather(batch_y, index) * (1 - lam_y)
    return mixed_x, mixed_y

def cutmix(batch_x, batch_y, alpha=1.0):
    lam = sample_beta_distribution(1, alpha)[0]
    batch_size, h, w, _ = batch_x.shape
    rx, ry = np.random.randint(w), np.random.randint(h)
    rw, rh = int(w * np.sqrt(1 - lam)), int(h * np.sqrt(1 - lam))
    x1, y1 = np.clip(rx - rw // 2, 0, w), np.clip(ry - rh // 2, 0, h)
    x2, y2 = np.clip(rx + rw // 2, 0, w), np.clip(ry + rh // 2, 0, h)
    index = tf.random.shuffle(tf.range(batch_size))
    batch_x[:, y1:y2, x1:x2, :] = tf.gather(batch_x, index)[:, y1:y2, x1:x2, :]
    lam = 1 - ((x2 - x1) * (y2 - y1) / (w * h))
    mixed_y = batch_y * lam + tf.gather(batch_y, index) * (1 - lam)
    return batch_x, mixed_y
def mixup_cutmix_generator(generator, use_cutmix=False):
    for batch_x, batch_y in generator:
        if np.random.rand() < 0.5:
            batch_x, batch_y = mixup(batch_x, batch_y)
        elif use_cutmix:
            batch_x, batch_y = cutmix(batch_x, batch_y)
        yield batch_x, batch_y
train_gen = mixup_cutmix_generator(train, use_cutmix=True)

In [ ]:
steps_per_epoch = len(train) 

cosine_lr = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=1e-4,
    first_decay_steps=steps_per_epoch * 10,  # restart period
    t_mul=1.5,
    m_mul=0.9,
    alpha=1e-5
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=cosine_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:

history = model.fit(
    train_gen,
    steps_per_epoch=len(train),
    validation_data=val,
    epochs=50
)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage
import matplotlib.pyplot as plt

# ------------------------------
# Helper: load image
# ------------------------------
def load_image(path, target_size=(256,256)):
    img = kimage.load_img(path, target_size=target_size)
    x = kimage.img_to_array(img) / 255.0
    return x

# ------------------------------
# Grad-CAM
# ------------------------------
def get_last_conv_layer(model):
    for layer in reversed(model.layers):
        try:
            # check if output is 4D (batch, H, W, C)
            if len(layer.output.shape) == 4:
                return layer.name
        except:
            continue
    raise ValueError("No conv layer found in the model.")

def grad_cam(model, img_tensor, class_index=None, last_conv_layer_name=None):
    if last_conv_layer_name is None:
        last_conv_layer_name = get_last_conv_layer(model)

    img_batch = np.expand_dims(img_tensor, axis=0)
    grad_model = tf.keras.models.Model([model.inputs],
                                       [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_batch)
        if class_index is None:
            class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0].numpy()

    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i].numpy()

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    if np.max(heatmap) > 0:
        heatmap /= np.max(heatmap)

    heatmap = tf.image.resize(heatmap[..., np.newaxis], (img_tensor.shape[0], img_tensor.shape[1])).numpy().squeeze()
    return heatmap

def overlay_heatmap(img, heatmap, alpha=0.4, cmap='jet'):
    import matplotlib.cm as cm
    jet = cm.get_cmap(cmap)
    heatmap_colored = jet(heatmap)[:,:,:3]
    overlay = img * (1 - alpha) + heatmap_colored * alpha
    return np.clip(overlay, 0, 1)

# ------------------------------
# Attention Rollout (analogue for Conv models)
# ------------------------------

def feature_map_rollout(model, img_tensor, conv_layer_names=None):
    if conv_layer_names is None:
        conv_layer_names = []
        for l in model.layers:
            try:
                if len(l.output.shape) == 4:  # 4D feature map
                    conv_layer_names.append(l.name)
            except:
                continue

    # Expand input batch
    img_batch = np.expand_dims(img_tensor, axis=0)

    feature_maps = []
    for layer_name in conv_layer_names:
        intermediate_model = tf.keras.Model(
            inputs=model.input,
            outputs=model.get_layer(layer_name).output
        )
        fmap = intermediate_model(img_batch)
        fmap = tf.reduce_mean(fmap, axis=-1).numpy()[0]  # avg across channels
        fmap = cv2.resize(fmap, (img_tensor.shape[1], img_tensor.shape[0]))
        feature_maps.append(fmap)

    # Rollout = average of all feature maps
    rollout = np.mean(feature_maps, axis=0)
    rollout = np.maximum(rollout, 0)
    rollout /= (rollout.max() + 1e-8)
    return rollout


In [ ]:
# After training is complete, just use the trained model in memory
# (the 'model' variable from model.fit)

sample_path_01 = "/kaggle/working/Train_test_splited/test/dyed-lifted-polyps/0bf20651-4369-41e5-b62d-7689a682e994.jpg"
img = load_image(sample_path_01)

# Grad-CAM
gradcam_map = grad_cam(model, img)
gradcam_overlay = overlay_heatmap(img, gradcam_map)

# Rollout
rollout_map = feature_map_rollout(model, img)
rollout_overlay = overlay_heatmap(img, rollout_map)

# Show results
plt.figure(figsize=(12,6))
plt.subplot(2,3,1); plt.title("Original"); plt.imshow(img); plt.axis('off')
plt.subplot(2,3,2); plt.title("Grad-CAM"); plt.imshow(gradcam_map, cmap='jet'); plt.axis('off')
plt.subplot(2,3,3); plt.title("Grad-CAM Overlay"); plt.imshow(gradcam_overlay); plt.axis('off')
plt.subplot(2,3,4); plt.title("Rollout"); plt.imshow(rollout_map, cmap='jet'); plt.axis('off')
plt.subplot(2,3,5); plt.title("Rollout Overlay"); plt.imshow(rollout_overlay); plt.axis('off')
plt.show()


In [ ]:
# After training is complete, just use the trained model in memory
# (the 'model' variable from model.fit)

sample_path_06 = "/kaggle/working/Train_test_splited/test/polyps/1ec57a41-f50e-4be1-a202-0de24c9d084b.jpg"
img = load_image(sample_path_06)

# Grad-CAM
gradcam_map = grad_cam(model, img)
gradcam_overlay = overlay_heatmap(img, gradcam_map)

# Rollout
rollout_map = feature_map_rollout(model, img)
rollout_overlay = overlay_heatmap(img, rollout_map)

# Show results
plt.figure(figsize=(12,6))
plt.subplot(2,3,1); plt.title("Original"); plt.imshow(img); plt.axis('off')
plt.subplot(2,3,2); plt.title("Grad-CAM"); plt.imshow(gradcam_map, cmap='jet'); plt.axis('off')
plt.subplot(2,3,3); plt.title("Grad-CAM Overlay"); plt.imshow(gradcam_overlay); plt.axis('off')
plt.subplot(2,3,4); plt.title("Rollout"); plt.imshow(rollout_map, cmap='jet'); plt.axis('off')
plt.subplot(2,3,5); plt.title("Rollout Overlay"); plt.imshow(rollout_overlay); plt.axis('off')
plt.show()


# **True prediction vs incorrect prediction**

In [ ]:
import random

# --- Step 1: Run predictions on test set ---
y_true = test.classes
class_indices = {v: k for k, v in test.class_indices.items()}
y_pred = model.predict(test, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# --- Step 2: Find correct & incorrect predictions ---
correct_idx = np.where(y_pred_classes == y_true)[0]
incorrect_idx = np.where(y_pred_classes != y_true)[0]

print(f"Correct predictions: {len(correct_idx)}, Incorrect: {len(incorrect_idx)}")

# Pick at least 3 from each (if available)
correct_samples = random.sample(list(correct_idx), min(3, len(correct_idx)))
incorrect_samples = random.sample(list(incorrect_idx), min(3, len(incorrect_idx)))


# --- Step 3: Visualization function ---
def visualize_prediction(idx, correct=True):
    img_path = test.filepaths[idx]
    img = load_image(img_path)

    true_label = class_indices[y_true[idx]]
    pred_label = class_indices[y_pred_classes[idx]]

    plt.imshow(img)
    plt.axis("off")
    plt.title(
        f"True: {true_label} | Pred: {pred_label}",
        color=("green" if correct else "red"),
        fontsize=12
    )
    plt.show()


# --- Step 4: Show 3 correct + 3 incorrect ---
print("\n=== Correctly Classified Samples ===")
for idx in correct_samples:
    visualize_prediction(idx, correct=True)

print("\n=== Incorrectly Classified Samples ===")
for idx in incorrect_samples:
    visualize_prediction(idx, correct=False)


In [ ]:
test_auc,test_loss=model.evaluate(test)

# **softmax confidence maps**

In [ ]:
import matplotlib.cm as cm

def confidence_map(model, img_tensor):
    """
    Generate softmax confidence map for the predicted class.
    Uses class activation backprop to highlight regions of high/low confidence.
    """
    img_batch = np.expand_dims(img_tensor, axis=0)

    # Get logits (before softmax)
    preds = model(img_batch, training=False)
    probs = tf.nn.softmax(preds[0]).numpy()
    pred_class = np.argmax(probs)
    confidence = probs[pred_class]

    # Gradients for confidence backprojection
    last_conv_layer = get_last_conv_layer(model)
    grad_model = tf.keras.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, predictions = grad_model(img_batch)
        loss = predictions[:, pred_class]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_out = conv_out[0].numpy()
    pooled_grads = pooled_grads.numpy()

    # Weight conv maps
    for i in range(conv_out.shape[-1]):
        conv_out[:, :, i] *= pooled_grads[i]

    # Normalize
    heatmap = np.mean(conv_out, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (heatmap.max() + 1e-8)

    # Resize to input size
    heatmap = cv2.resize(heatmap, (img_tensor.shape[1], img_tensor.shape[0]))

    return heatmap, confidence, pred_class, probs


def visualize_confidence(img_path):
    img = load_image(img_path)
    heatmap, conf, pred_class, probs = confidence_map(model, img)

    plt.figure(figsize=(10,5))
    plt.subplot(1,3,1); plt.title("Original"); plt.imshow(img); plt.axis("off")
    plt.subplot(1,3,2); plt.title(f"Confidence Heatmap\nConf: {conf:.2f}")
    plt.imshow(heatmap, cmap="jet"); plt.axis("off")
    plt.subplot(1,3,3); plt.title("Overlay")
    plt.imshow(img); plt.imshow(heatmap, cmap="jet", alpha=0.5); plt.axis("off")
    plt.show()

    print("Class probabilities:", probs)


# Example on a test sample
sample_idx = random.choice(range(len(test.filepaths)))
visualize_confidence(test.filepaths[sample_idx])


# **Quantitative Metrics on test set**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# --- Predictions on test set ---
y_true = test.classes
class_indices = {v: k for k, v in test.class_indices.items()}
y_pred_logits = model.predict(test, verbose=1)
y_pred_probs = tf.nn.softmax(y_pred_logits, axis=1).numpy()
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# --- Basic Metrics ---
acc = accuracy_score(y_true, y_pred_classes)
prec = precision_score(y_true, y_pred_classes, average="macro")
rec = recall_score(y_true, y_pred_classes, average="macro")
f1 = f1_score(y_true, y_pred_classes, average="macro")

# One-vs-rest AUC (multi-class)
try:
    auc = roc_auc_score(y_true, y_pred_probs, multi_class="ovr")
except:
    auc = None

print("=== Performance Metrics on Test Set ===")
print(f"Accuracy   : {acc:.4f}")
print(f"Precision  : {prec:.4f}")
print(f"Recall     : {rec:.4f}")
print(f"F1-score   : {f1:.4f}")
if auc is not None:
    print(f"AUC        : {auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_indices.values()))


# --- Calibration Metrics (ECE & MCE) ---
def calibration_metrics(probs, labels, n_bins=15):
    """
    Expected Calibration Error (ECE) and Maximum Calibration Error (MCE)
    """
    confidences = np.max(probs, axis=1)
    predictions = np.argmax(probs, axis=1)

    ece = 0.0
    mce = 0.0
    bin_boundaries = np.linspace(0.0, 1.0, n_bins + 1)

    for i in range(n_bins):
        start = bin_boundaries[i]
        end = bin_boundaries[i+1]
        mask = (confidences > start) & (confidences <= end)
        if np.any(mask):
            acc_bin = np.mean(predictions[mask] == labels[mask])
            conf_bin = np.mean(confidences[mask])
            gap = abs(acc_bin - conf_bin)
            ece += (np.sum(mask) / len(labels)) * gap
            mce = max(mce, gap)

    return ece, mce

ece, mce = calibration_metrics(y_pred_probs, y_true)
print("\n=== Calibration Metrics ===")
print(f"ECE (Expected Calibration Error): {ece:.4f}")
print(f"MCE (Maximum Calibration Error) : {mce:.4f}")


# ROC CURVE

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

# --- Binarize labels for ROC ---
n_classes = len(class_indices)
y_true_bin = label_binarize(y_true, classes=list(range(n_classes)))

# --- ROC Curve for each class ---
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# --- Macro-average ROC ---
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# --- Plot ---
plt.figure(figsize=(8,6))

# Per-class curves
for i, class_name in class_indices.items():
    plt.plot(fpr[i], tpr[i],
             label=f"Class {class_name} (AUC = {roc_auc[i]:.2f})")

# Macro-average curve
plt.plot(fpr["macro"], tpr["macro"],
         label=f"Macro-average (AUC = {roc_auc['macro']:.2f})",
         color="navy", linestyle="--", linewidth=2)

plt.plot([0, 1], [0, 1], "k--", lw=2)  # diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (One-vs-Rest)")
plt.legend(loc="lower right")
plt.show()
